In [1]:
# 메모리내 변수 제거

all = [var for var in globals() if var[0] != "_"]   # globals() 목록의 첫글자가 _ 로 시작하지 않는 자료의 리스트만 가져와서
for var in all:
    del globals()[var]

In [2]:
#%pip install xgboost
#%pip install wordcloud
#%pip install gensim

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics import accuracy_score
## for data
import pandas as pd

import re
import nltk## for language detection

# 데이터 수집

In [4]:
train = pd.read_csv('c:/data/project/train.csv')
test = pd.read_csv('c:/data/project/test.csv')

train.head()

,ID,first_party,second_party,facts,first_party_winner
0,TRAIN_0000,Phil A. St. Amant,Herman A. Thompson,"On June 27, 1962, Phil St. Amant, a candidate ...",1
1,TRAIN_0001,Stephen Duncan,Lawrence Owens,Ramon Nelson was riding his bike when he suffe...,0
2,TRAIN_0002,Billy Joe Magwood,"Tony Patterson, Warden, et al.",An Alabama state court convicted Billy Joe Mag...,1
3,TRAIN_0003,Linkletter,Walker,Victor Linkletter was convicted in state court...,0
4,TRAIN_0004,William Earl Fikes,Alabama,"On April 24, 1953 in Selma, Alabama, an intrud...",1


# 데이터 전처리

In [5]:
df = train[['facts', 'first_party_winner']]
df_target = df[['first_party_winner']]
df_nlp = df[['facts']]
df_nlp1 = pd.DataFrame(df_nlp, columns=['facts'])
df_nlp1['facts'] = df_nlp1['facts'].str.replace(r'<[^<>]*>', '', regex=True) # 특수 문자 제거
df_nlp1['facts'] = df_nlp1['facts'].str.replace(r'\d', '', regex=True)  # 숫자 제거

In [6]:
corpus = df_nlp1['facts']
# print(corpus.str.cat(sep=" ")) # 인덱스의 요소들 서로 잇기
lst_tokens = nltk.tokenize.word_tokenize(corpus.str.cat(sep=" "))
ps = nltk.stem.porter.PorterStemmer()
lem = nltk.stem.wordnet.WordNetLemmatizer()
lst_stopwords = nltk.corpus.stopwords.words("english")

In [7]:
# 문장 깨끗하게 만드는 함수

def utils_preprocess_text(text, flg_stemm=True, flg_lemm=True, lst_stopwords=None):
    ## clean (convert to lowercase and remove punctuations and characters and then strip)
    import re
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
    
    ## Tokenize (convert from string to list)
    lst_text = text.split()    ## remove Stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in 
                    lst_stopwords]
                
    ## Stemming (remove -ing, -ly, ...)
    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]
                
    ## Lemmatisation (convert the word into root word)
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]
            
    ## back to string from list # 역토큰화, 벡터화 하기 위해서
    text = " ".join(lst_text)
    return text

In [8]:
# 문장 깨끗하게 만드는 함수 사용
df_nlp1["facts_clean"] = df_nlp1["facts"].apply(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=True, lst_stopwords=lst_stopwords))
corpus_stopwords = df_nlp1["facts_clean"]
lst_tokens_stopwords = nltk.tokenize.word_tokenize(corpus_stopwords.str.cat(sep=" "))
stop_words=[]
for word, freq in nltk.FreqDist(lst_tokens_stopwords).most_common():
    if freq == 1:
        #print(word)
        stop_words.append(word)

df_nlp1["facts_clean"] = df_nlp1["facts_clean"] .apply(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=False, lst_stopwords=stop_words))
# y값 포함해서 하나의 프레임으로 만들기
df_nlp2 = pd.concat([df_nlp1,df_target['first_party_winner']],axis=1, join='inner')
df_nlp2 = df_nlp2.drop(columns='facts')
df_nlp2

,facts_clean,first_party_winner
0,june phil st amant candidate public office mad...,1
1,ramon nelson riding bike suffered lethal blow ...,0
2,alabama state court convicted billy joe magwoo...,1
3,victor linkletter convicted state court eviden...,0
4,april selma alabama intruder broke apartment d...,1
...,...,...
2473,congress amended clean air act energy policy a...,1
2474,alliance bond fund inc investment fund purchas...,1
2475,district court sentenced manuel peguero month ...,0
2476,march st cyr lawful permanent resident pled gu...,0


In [9]:
#TFIDF 적용, 문서들의 유사도를 구하는 작업

# vectorizeTF = TfidfVectorizer()
# count_matrix_tf = vectorizeTF.fit_transform(df_nlp2['facts_clean'])
# data_final = count_matrix_tf.toarray()
# data_final = pd.DataFrame(data=data_final, columns=vectorizeTF.get_feature_names_out())
# data_final = pd.concat([data_final,df_nlp2["first_party_winner"]],axis=1,join='inner')
# data_final.vocabulary_

In [10]:
# 카운터 벡터라이즈는 사용중단
vectorizeCV=CountVectorizer()
count_matrix = vectorizeCV.fit_transform(df_nlp2['facts_clean'])
count_array = count_matrix.toarray()
data_final = pd.DataFrame(data=count_array,columns = vectorizeCV.get_feature_names_out())
data_final = pd.concat([data_final,df_nlp2["first_party_winner"]],axis=1,join='inner')
data_final

,__,aa,aacw,aai,aar,aaron,ab,abandoned,abandonment,abatement,...,zj,zone,zoned,zoneofinterests,zoning,zuni,zurich,zurko,zurkos,first_party_winner
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2473,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2474,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2475,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2476,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# 데이터 모델링

In [11]:
X_train, X_test, y_train, y_test = train_test_split(data_final.drop(columns=['first_party_winner']), data_final['first_party_winner'], test_size=0.3,random_state=0)

In [12]:
# LDA 학습, 단어의 의미구조 파악
# 30초정도 걸림
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=200, random_state=1) # n_component : 토픽 갯수
lda_data = lda.fit_transform(X_train)

In [13]:
# LDA 학습 결과 보기
# terms = vectorizeTF.get_feature_names_out()
terms = vectorizeCV.get_feature_names_out()
def get_topics(components, feature_names, n=5): # n: n개 출력
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])

get_topics(lda.components_,terms)

# 토픽과 토픽에 대한 기여도

Topic 1: [('court', 23.46), ('car', 15.81), ('byrd', 12.0), ('davila', 11.0), ('basic', 11.0)]
Topic 2: [('court', 15.36), ('property', 13.89), ('state', 11.29), ('land', 10.37), ('claim', 9.39)]
Topic 3: [('court', 34.94), ('act', 20.57), ('circuit', 11.56), ('federal', 11.38), ('employee', 9.13)]
Topic 4: [('patent', 87.69), ('infringement', 25.76), ('district', 17.25), ('inc', 11.36), ('akamai', 9.0)]
Topic 5: [('court', 37.41), ('security', 17.48), ('appeal', 14.21), ('bm', 14.0), ('act', 13.36)]
Topic 6: [('court', 39.94), ('act', 14.69), ('district', 13.29), ('law', 12.76), ('circuit', 11.58)]
Topic 7: [('court', 37.4), ('debt', 17.0), ('federal', 15.37), ('appeal', 14.16), ('florida', 13.84)]
Topic 8: [('court', 26.93), ('leave', 11.9), ('state', 11.71), ('appeal', 7.66), ('law', 7.0)]
Topic 9: [('trust', 19.81), ('tax', 18.63), ('court', 15.7), ('state', 14.74), ('aca', 11.78)]
Topic 10: [('court', 13.27), ('rate', 13.0), ('oil', 11.9), ('gas', 11.19), ('provider', 11.0)]
Topic

In [14]:
# XG부스트 모델 적용하기

# param_dict = {'max_depth': 10, 'min_child_weight': 1, 'n_estimators': 300}
# lda_data = X_train
# lda = vectorizeTF
model = XGBClassifier() # **param_dict
x = pd.DataFrame(data=lda_data)
model.fit(x, y_train)

# 테스트 lda 적용
xtest = pd.DataFrame(data=lda.transform(X_test))

# 모델로 예측해보기
y_pred = model.predict(xtest)
predictions = [round(value) for value in y_pred]

In [15]:
# 평가

from sklearn.metrics import f1_score
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
f1_score(y_test, y_pred)

Accuracy: 58.33%


0.7166361974405849

In [16]:
# 최적의 파라미터 값 찾아보기
# 3분 47초
from sklearn.model_selection import GridSearchCV

def findParam():
    param_xgb = {"max_depth": [10,30,50],
                "min_child_weight" : [1,3,6,10],
                "n_estimators": [200,300,500,1000]
                }    

    gscv_xgb = GridSearchCV (estimator = model, param_grid = param_xgb, scoring ='accuracy', cv = 3, refit=True, n_jobs=1, verbose=2)
    gscv_xgb.fit(x, y_train)

    print('XGB 파라미터: ', gscv_xgb.best_params_)
    print('XGB 예측 정확도: {:.4f}'.format(gscv_xgb.best_score_))

# XGB 파라미터:  {'max_depth': 10, 'min_child_weight': 1, 'n_estimators': 300}
# XGB 예측 정확도: 0.6044

# 테스트csv 적용

In [17]:
dfTest = pd.DataFrame(test['facts'])
dfTest['facts'] = dfTest['facts'].str.replace(r'<[^<>]*>', '', regex=True)
dfTest['facts'] = dfTest['facts'].str.replace(r'\d', '', regex=True)  # 숫자 제거
dfTest["facts_clean"] = dfTest["facts"].apply(lambda x: utils_preprocess_text(x, flg_stemm=False, flg_lemm=True, lst_stopwords=lst_stopwords))
dfTest['first_party_winner'] = np.zeros(len(dfTest)).astype(int)
dfTest

,facts,facts_clean,first_party_winner
0,The Bail Reform Act allowed the federal court...,bail reform act allowed federal court detain a...,0
1,Lexecon Inc. was a defendant in a class action...,lexecon inc defendant class action lawsuit usc...,0
2,"In and , Fox Television Stations broadcast th...",fox television station broadcast billboard mus...,0
3,During his trial for armed robbery of a federa...,trial armed robbery federally insured saving l...,0
4,"In , a magistrate judge issued a warrant autho...",magistrate judge issued warrant authorizing se...,0
...,...,...,...
1235,According to Executive Order No. signed by Pr...,according executive order signed president geo...,0
1236,Section (a) of the Clean Air Act (CAA) require...,section clean air act caa requires environment...,0
1237,Linda Matteo and John Madigan created a plan f...,linda matteo john madigan created plan utilizi...,0
1238,"In , the North Carolina Board of Agriculture a...",north carolina board agriculture adopted regul...,0


In [18]:
# count_matrix_test = vectorizeTF.transform(dfTest['facts_clean'])
# count_array_test = count_matrix_test.toarray()
# data_final_test = pd.DataFrame(data=count_array_test,columns = vectorizeTF.get_feature_names_out())
# data_final_test = pd.concat([data_final_test,dfTest["first_party_winner"]],axis=1,join='inner')
# data_final_test

In [19]:
count_matrix_test = vectorizeCV.transform(dfTest['facts_clean'])
count_array_test = count_matrix_test.toarray()
data_final_test = pd.DataFrame(data=count_array_test,columns = vectorizeCV.get_feature_names_out())
data_final_test = pd.concat([data_final_test,dfTest["first_party_winner"]],axis=1,join='inner')
data_final_test

,__,aa,aacw,aai,aar,aaron,ab,abandoned,abandonment,abatement,...,zj,zone,zoned,zoneofinterests,zoning,zuni,zurich,zurko,zurkos,first_party_winner
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1235,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1236,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1237,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1238,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
X_test = data_final_test.drop(columns=['first_party_winner'])
y_test = data_final_test['first_party_winner']

In [21]:
x_lda_test = pd.DataFrame(data=lda.transform(X_test))
y_pred_test = model.predict(x_lda_test)
predictionsTest = [round(value) for value in y_pred_test]
predcsv = pd.DataFrame(predictionsTest,columns=['first_party_winner'])

In [22]:
get_topics(lda.components_,terms)

Topic 1: [('court', 23.46), ('car', 15.81), ('byrd', 12.0), ('davila', 11.0), ('basic', 11.0)]
Topic 2: [('court', 15.36), ('property', 13.89), ('state', 11.29), ('land', 10.37), ('claim', 9.39)]
Topic 3: [('court', 34.94), ('act', 20.57), ('circuit', 11.56), ('federal', 11.38), ('employee', 9.13)]
Topic 4: [('patent', 87.69), ('infringement', 25.76), ('district', 17.25), ('inc', 11.36), ('akamai', 9.0)]
Topic 5: [('court', 37.41), ('security', 17.48), ('appeal', 14.21), ('bm', 14.0), ('act', 13.36)]
Topic 6: [('court', 39.94), ('act', 14.69), ('district', 13.29), ('law', 12.76), ('circuit', 11.58)]
Topic 7: [('court', 37.4), ('debt', 17.0), ('federal', 15.37), ('appeal', 14.16), ('florida', 13.84)]
Topic 8: [('court', 26.93), ('leave', 11.9), ('state', 11.71), ('appeal', 7.66), ('law', 7.0)]
Topic 9: [('trust', 19.81), ('tax', 18.63), ('court', 15.7), ('state', 14.74), ('aca', 11.78)]
Topic 10: [('court', 13.27), ('rate', 13.0), ('oil', 11.9), ('gas', 11.19), ('provider', 11.0)]
Topic

In [23]:
submit = pd.read_csv('C:/data/project/sample_submission.csv')
submit['first_party_winner'] = predcsv
submit.to_csv('./sample_submission.csv', index=False)
print('Done')

Done


In [24]:
predcsv.value_counts()

first_party_winner
1                     1004
0                      236
Name: count, dtype: int64